In [1]:
#installing missing library
!pip install transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 25.8 MB/s 
     |████████████████████████████████| 6.6 MB 48.4 MB/s 
     |████████████████████████████████| 120 kB 61.9 MB/s 


In [2]:
from transformers import AutoTokenizer, AutoModel
import torch

In [3]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/stsb-distilbert-base')
model = AutoModel.from_pretrained('sentence-transformers/stsb-distilbert-base')

In [4]:
text = "hello world!"

tokens = tokenizer.encode_plus(text, max_length=128,
                               truncation=True, padding='max_length',
                               return_tensors='pt')

In [5]:
outputs = model(**tokens)
outputs

BaseModelOutput(last_hidden_state=tensor([[[-1.5872,  0.4290, -0.6303,  ..., -0.2767,  0.7593,  0.5782],
         [-1.5122,  0.8271, -0.3605,  ...,  0.0248,  0.9586,  0.2221],
         [-1.1622,  0.1706, -0.2628,  ..., -0.0566,  0.6829,  0.3858],
         ...,
         [-1.6612,  0.5686, -0.6592,  ...,  0.0665,  0.2652,  0.6174],
         [-1.5050,  0.4754, -0.5813,  ...,  0.1066,  0.6722,  0.5357],
         [-1.7271,  0.2002, -0.5729,  ...,  0.2163,  0.7831,  0.5169]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)

In [6]:
embeddings = outputs.last_hidden_state
embeddings

tensor([[[-1.5872,  0.4290, -0.6303,  ..., -0.2767,  0.7593,  0.5782],
         [-1.5122,  0.8271, -0.3605,  ...,  0.0248,  0.9586,  0.2221],
         [-1.1622,  0.1706, -0.2628,  ..., -0.0566,  0.6829,  0.3858],
         ...,
         [-1.6612,  0.5686, -0.6592,  ...,  0.0665,  0.2652,  0.6174],
         [-1.5050,  0.4754, -0.5813,  ...,  0.1066,  0.6722,  0.5357],
         [-1.7271,  0.2002, -0.5729,  ...,  0.2163,  0.7831,  0.5169]]],
       grad_fn=<NativeLayerNormBackward0>)

In [7]:
embeddings.shape

torch.Size([1, 128, 768])

In [9]:
'''
After we have produced our dense vectors embeddings
we need to perform a mean pooling operation on them to create a single vector encoding (the sentence embedding). 
To do this mean pooling operation we will need to multiply each value in our embeddings tensor by it's respective attention_mask value
- so that we ignore non-real tokens.
'''

"\nAfter we have produced our dense vectors embeddings\nwe need to perform a mean pooling operation on them to create a single vector encoding (the sentence embedding). \nTo do this mean pooling operation we will need to multiply each value in our embeddings tensor by it's respective attention_mask value\n- so that we ignore non-real tokens.\n"

In [8]:
attention_mask = tokens['attention_mask']
attention_mask.shape

torch.Size([1, 128])

In [10]:
mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
mask.shape

torch.Size([1, 128, 768])

In [11]:
attention_mask

tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])

In [12]:
mask[0][0].shape

torch.Size([768])

In [ ]:
#Each vector below represents a single token attention mask - each token now has a vector of size 768 representing it's attention_mask status.

In [13]:
mask

tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])

In [14]:
#applying the attention mask by multiplying 2 tensors 
masked_embeddings = embeddings * mask
masked_embeddings.shape

torch.Size([1, 128, 768])

In [15]:
masked_embeddings

tensor([[[-1.5872,  0.4290, -0.6303,  ..., -0.2767,  0.7593,  0.5782],
         [-1.5122,  0.8271, -0.3605,  ...,  0.0248,  0.9586,  0.2221],
         [-1.1622,  0.1706, -0.2628,  ..., -0.0566,  0.6829,  0.3858],
         ...,
         [-0.0000,  0.0000, -0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0000,  0.0000, -0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0000,  0.0000, -0.0000,  ...,  0.0000,  0.0000,  0.0000]]],
       grad_fn=<MulBackward0>)

In [16]:
summed = torch.sum(masked_embeddings, 1)
summed.shape

torch.Size([1, 768])

In [17]:
summed = torch.sum(masked_embeddings, 1)
summed.shape

torch.Size([1, 768])

In [19]:
summed_mask = torch.clamp(mask.sum(1), min=1e-9)
summed_mask.shape

torch.Size([1, 768])

In [20]:
summed_mask

tensor([[5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
         5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
         5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
         5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
         5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
         5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
         5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
         5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
         5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
         5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
         5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
         5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
         5., 5., 5., 5., 5.,

In [22]:
mean_pooled = summed / summed_mask

In [23]:
mean_pooled

tensor([[-1.3574e+00,  4.0033e-01, -5.0986e-01, -9.4546e-01, -8.4032e-01,
          2.3364e-01,  3.3641e-01,  1.2409e+00, -4.5858e-01, -1.8770e-01,
         -2.2717e-01,  3.3995e-01, -5.5674e-01,  5.0741e-01, -3.4537e-01,
          1.6458e-02, -1.0587e+00,  3.2711e-01, -9.2551e-01,  2.2697e-01,
         -2.1575e-02, -8.4158e-01,  1.0492e-01, -3.9943e-01, -2.3523e-01,
         -1.9869e-01,  5.6704e-01,  6.6077e-01, -4.9127e-01, -3.1601e-01,
         -5.8040e-01, -8.3447e-03,  8.8351e-01,  5.7043e-01, -8.6027e-01,
          4.9987e-01, -1.8409e+00, -3.2400e-01, -6.1466e-01,  5.2581e-01,
          6.1306e-01, -4.6464e-01,  2.4222e-01, -7.2605e-01, -7.0671e-01,
          1.1250e-01, -1.0787e+00,  4.0922e-01, -8.1751e-01, -1.0727e-01,
         -1.0153e-02,  4.9474e-01,  4.1710e-01, -1.3323e-01, -3.3978e-01,
         -1.2359e-01, -1.3152e+00,  5.8368e-01,  2.8340e-01, -5.1474e-01,
         -8.6234e-01, -8.5961e-01,  6.1817e-01,  4.8220e-01,  1.2305e-02,
         -3.4793e-02,  6.6903e-01,  6.